In [1]:
using JuMP, Cbc

nodes = 1:4
transshipment_nodes = 2:3
arcs = [(1,2), (1,3), (2,3), (2,4), (3,4)]
U = Dict()
for (i,j) in arcs
    U[i,j]=Inf
end
b = [10, 0, 0, -10]
c = Dict(zip(arcs, [3, 2, 2, 5, 1]))
u = [0, 6, 6, 0]

m = Model()

@variable(m, 0 <= x[a in arcs] <= U[a])

@objective(m, Min, sum(c[a]x[a] for a in arcs))

@constraint(m, flowbalance[i in nodes],
    sum(x[(i,j)] for j in nodes if (i,j) in arcs) - 
    sum(x[(j,i)] for j in nodes if (j,i) in arcs) 
    == b[i])
@constraint(m, nodecapacity[i in transshipment_nodes],
    sum(x[(j,i)] for j in nodes if (j,i) in arcs) 
    <= u[i])

set_optimizer(m, Cbc.Optimizer)
optimize!(m)

println("Mininum cost = ", round(objective_value(m),digits=1))
for a in arcs
    println("flow on arc ", a, "= ", round(value(x[a]),digits=1))
end

Mininum cost = 50.0
flow on arc (1, 2)= 4.0
flow on arc (1, 3)= 6.0
flow on arc (2, 3)= 0.0
flow on arc (2, 4)= 4.0
flow on arc (3, 4)= 6.0
Presolve 3 (-3) rows, 3 (-2) columns and 8 (-5) elements
0  Obj 41.499999 Primal inf 4.0999973 (3)
3  Obj 50
Optimal - objective value 50
After Postsolve, objective 50, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 50 - 3 iterations time 0.002, Presolve 0.00
